In [1]:
import datetime
import mibian
import datetime as dt
from datetime import date, timedelta

import pymysql
import pandas as pd
import numpy as np


import csv
import json
import pymysql

import requests
from time import gmtime, strftime
import pandas_datareader as pdr
import datetime as dt
#connecting to database
conn=pymysql.connect(host='148.66.134.234',port=int(3306),user='unhotel_user',db='unhotel_nodal_data_trading',passwd='Nodal@123456')

df=pd.read_sql_query("SELECT * FROM rapid_api_option_chain ORDER BY ID DESC LIMIT 10000",conn) #fetching latest data

todays_time=dt.datetime.today()
yesterday_time=todays_time-dt.timedelta(hours=todays_time.hour,minutes=todays_time.minute)
day_bef_yesterday=yesterday_time-dt.timedelta(days=1)
todays_df=df[(df['timestamp'] > str(yesterday_time)) & (df['timestamp'] < str(todays_time) )]
todays_df.reset_index(inplace=True)
todays_df.drop(['index'],inplace=True,axis=1)
#creating yesterdays_df
yesterday_df=df[(df['timestamp'] > str(day_bef_yesterday)) & (df['timestamp'] < str(yesterday_time) )]
yesterday_df.reset_index(inplace=True)
yesterday_df.drop(['index'],inplace=True,axis=1)
#check if todays df is null
if todays_df.shape[0] ==0:
    
    todays_df=yesterday_df
def roundoff_underlying_symbol(ticker,df):
    df_new=df[df['ce_underlying']==ticker].reset_index()
    df_new.drop(['index'],inplace=True,axis=1)
    return roundoff_underlying(df_new,ticker)
    
# def strike_dif(df):
#     return int(float(df['ce_strikeprice'].unique()[1]))-int(float(df['ce_strikeprice'].unique()[0]))
def roundoff_underlying(df,ticker):
#     a=int(float(df['ce_underlyingvalue'][0]))
#     if(strike_dif(df)<=10):
#         return(round(a,-1))
#     elif(strike_dif(df)>10 and strike_dif(df)<=200):
#         return(round(a,-2))
#     else:
#         return(round(a,-3))
    df.reset_index(drop=True,inplace=True)
    mylist=list(df['strikeprice'])
    mylist=list(map(float, mylist))
    myNumber=int(df[df['ce_underlying']==ticker]['pe_underlyingvalue'].astype(float).mean())
    return min(mylist, key=lambda x:abs(x-myNumber))

#list of stocks
symbol=['ADANIPORTS', 'ASIANPAINT', 'AUBANK', 'AXISBANK', 'BAJAJ-AUTO', 'BAJAJFINSV', 'BAJFINANCE', 'BANDHANBNK', 'BHARTIARTL',
       'BPCL', 'BRITANNIA', 'CIPLA', 'COALINDIA', 'DIVISLAB', 'DRREDDY', 'EICHERMOT', 'FEDERALBNK', 'GRASIM', 'HCLTECH', 'HDFC',
       'HDFCBANK', 'HDFCLIFE', 'HEROMOTOCO', 'HINDALCO', 'HINDUNILVR', 'ICICIBANK', 'IDFCFIRSTB', 'INDUSINDBK', 'INFY', 'IOC', 
       'ITC', 'JSWSTEEL', 'KOTAKBANK', 'LT', 'MARUTI', 'NESTLEIND', 'NTPC', 'ONGC', 'PNB', 'POWERGRID', 'RBLBANK', 'RELIANCE', 
        'SBILIFE', 'SBIN', 'SHREECEM', 'SUNPHARMA','TATACONSUM', 'TATAMOTORS', 'TATASTEEL', 'TCS', 'TECHM', 'TITAN', 'ULTRACEMCO', 'UPL', 'WIPRO']

def show_dict(df):
    atm=[]
    for i in symbol:
        atm.append(roundoff_underlying_symbol(i,df))
    my_dict=dict(zip(symbol,atm))
    return my_dict
#give todays df with all values
todays_filter_df=pd.DataFrame()

abc=show_dict(todays_df)
for key,value in abc.items():
    
#   print(value)
    filter1=todays_df['ce_underlying']==key
    filter2=todays_df['strikeprice'].astype(float).astype(int)==(int(value))
    a=(todays_df.where(filter1 & filter2).dropna())
    todays_filter_df = pd.concat([todays_filter_df, a], axis=0,ignore_index=False).reset_index(drop=True)
    todays_filter_df.drop(['id'],inplace=True,axis=1)
    
#filter with latest expiry date
first_day_of_month=date.today().replace(day=1)
def last_day_of_month(any_day):
	# get close to the end of the month for any day, and add 4 days 'over'
    next_month = any_day.replace(day=28) + dt.timedelta(days=4)
	# subtract the number of remaining 'overage' days to get last day of current month, or said programattically said, the previous day of the first of next month
    return next_month - dt.timedelta(days=next_month.day)


last_day_of_month=last_day_of_month(dt.date.today())
last_thursday= pd.date_range(first_day_of_month, last_day_of_month, freq="W-THU").values[-1]
last_thursday=pd.to_datetime(last_thursday).date()
todays_latest_exp_df=todays_filter_df[pd.to_datetime(todays_filter_df['ce_expirydate'])==str(last_thursday)]
todays_latest_exp_df.reset_index(drop=True,inplace=True)

#calivol
a=float((todays_latest_exp_df['pe_underlyingvalue'][0]))
b=int(todays_latest_exp_df['pe_strikeprice'][0])
c=int(float(todays_latest_exp_df['pe_askprice'][0]))
def cal_ivol(df,i):
    
    a=float((df['pe_underlyingvalue'][i]))
    b=int(float(df['pe_strikeprice'][i]))
    c=int(float(df['pe_askprice'][i]))
    underlying_price = a
    #call_strike = 8700
    put_strike = b
    call_price = 669.90
    put_price = c
    interest_rate = 4 # % (as we are taking futures prices as underlying)
    days_to_expiry = 19
    p = mibian.BS([a, b, 4, 19], putPrice=c)
    return p.impliedVolatility


a=[]
for i in range(len(symbol)):
    a.append(cal_ivol(todays_latest_exp_df,i))
df_ivol=pd.DataFrame(a)
df_ivol.rename(columns ={0:'IVol'}, inplace =True)

#finding stradle
a=[]
for i in range(len(symbol)):
    a.append(float(todays_latest_exp_df['ce_bidprice'][i])+float(todays_latest_exp_df['ce_askprice'][i])/float(todays_latest_exp_df['ce_underlyingvalue'][i])*100)
df_Strad=pd.DataFrame(a)
df_Strad.rename(columns ={0:'Strad%'}, inplace =True)

#cal the Rvols
def get_n_days_data(ticker, x):
    #ticker = "AAPL"
    #start = dt.datetime(2021, 1, 1)
    #N_DAYS_AGO = 2
    n_days=150
    day = dt.datetime.today().weekday()
    
    end = dt.datetime.today()
    start = end - dt.timedelta(days=n_days)
    #diff=end-start
    data = pdr.get_data_yahoo(ticker, start, end)

    data['Log returns'] = np.log(data['Close']/data['Close'].shift())
    
    data['Log returns'].std()
    
    df=data[-x:]
#     volatility = data['Log returns'].std()*n_days**.5
    return df['Log returns'].std()*252**.5

RVol_3d=[]
RVol_7d=[]
RVol_25d=[]
RVol_45d=[]
RVol_100d=[]

for i in symbol:
    
    ticker=i+'.NS'
    RVol_3d.append(get_n_days_data(ticker,3))
    RVol_7d.append(get_n_days_data(ticker,7))
    RVol_25d.append(get_n_days_data(ticker,25))
    RVol_45d.append(get_n_days_data(ticker,45))
    RVol_100d.append(get_n_days_data(ticker,100))
    
    
RVol_df=pd.DataFrame()
RVol_3d= [round(num*100,2) for num in RVol_3d]
RVol_7d= [round(num*100,2) for num in RVol_7d]
RVol_25d = [round(num*100,2) for num in RVol_25d]
RVol_100d= [round(num*100,2) for num in RVol_100d]
RVol_45d= [round(num*100,2) for num in RVol_45d]

RVol_df['RVol_3d']=RVol_3d
RVol_df['RVol_7d']=RVol_7d
RVol_df['RVol_25d']=RVol_25d
RVol_df['RVol_45d']=RVol_45d
RVol_df['RVol_100d']=RVol_100d

cols=['ce_underlying',
 'strikeprice',
 'expirydate',
 'ce_strikeprice',
 'ce_underlyingvalue',
 'timestamp',
 'IVol',
 'Strad%',
 'RVol_3d',
 'RVol_7d',
 'RVol_25d',
 'RVol_45d',
 'RVol_100d']
todays_latest_exp_df=todays_latest_exp_df.drop_duplicates(subset='ce_underlying')
todays_latest_exp_df.reset_index(drop=True,inplace=True)
cols=['ce_underlying',
 'strikeprice',
 'expirydate',
 'ce_strikeprice',
 'ce_underlyingvalue',
 'timestamp',
 'IVol',
 'Strad%',
 'RVol_3d',
 'RVol_7d',
 'RVol_25d',
 'RVol_45d',
 'RVol_100d']
final=pd.concat([todays_latest_exp_df,df_ivol,df_Strad,RVol_df],axis=1)[cols]
print(final)
final.to_csv('final')


OperationalError: (1130, "45.249.87.54' is not allowed to connect to this MySQL server")

In [ ]:
df=pd.read_csv('final')


In [ ]:
todays_latest_exp_df